In [ ]:
pip install git+https://github.com/nengo-labs/nengo-gyrus

  Cloning https://github.com/nengo-labs/nengo-gyrus to /tmp/pip-req-build-6cxhbkml
  Running command git clone -q https://github.com/nengo-labs/nengo-gyrus /tmp/pip-req-build-6cxhbkml
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532kB 5.3MB/s 
  Created wheel for gyrus: filename=gyrus-0.1.0.dev1-cp36-none-any.whl size=31500 sha256=da40d13b8a3d8d9ea0cbdf6b9de770ed0ad94276d79cff74b082750d0fd10250
  Stored in directory: /tmp/pip-ephem-wheel-cache-9uxb2pt5/wheels/93/d6/58/e162c32fab63dd1d3ce90c57a2200e8ee1d6ab881c0e1fcda4
Successfully built gyrus


In [ ]:
pip install nengo-dl

     |████████████████████████████████| 286kB 5.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for nengo-dl: filename=nengo_dl-3.4.0-cp36-none-any.whl size=203449 sha256=0baed9eabd28b2dddc9cd6dd51cbc444afce6d817908890ac2968088e4e24149
  Stored in directory: /root/.cache/pip/wheels/ed/bc/2c/e4ab34a10be9a9feeb194f89ba2e00be6b995b4c73c9b90241
Successfully built nengo-dl
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
pip install nengo nengo-gui

     |████████████████████████████████| 849kB 5.5MB/s 
ERROR: nengo-gui 0.4.7 has requirement nengo<=3.0.0,>=2.6.0, but you'll have nengo 3.1.0 which is incompatible.


In [ ]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import nengo
import nengo_dl
import numpy as np
import tensorflow as tf
from matplotlib.colors import Normalize
from nengo.utils.numpy import rms
from nengo_gui.ipython import InlineGUI

import gyrus

In [ ]:
# load data
from numpy import loadtxt

DATA_FILE = '../../Data/100-Points_between_10-80_all'
dataset = loadtxt(DATA_FILE, delimiter=',')

dataset[:,3:8] = dataset[:,3:8] + 100*np.pi
dataset[:,3:8] = dataset[:,3:8] % (2*np.pi)

dataXyzNonNorm = []
dataAnglesNonNorm = []

START_FROM = 90
num_of_blocks = 1
num_of_points_in_block = 10

count = num_of_blocks*num_of_points_in_block
for i in range(count):
    dataXyzNonNorm.append(
        dataset[START_FROM + i*num_of_points_in_block:START_FROM + (i+1)*num_of_points_in_block,0:3]
    )
    dataAnglesNonNorm.append(
        dataset[START_FROM + i*num_of_points_in_block:START_FROM + (i+1)*num_of_points_in_block,3:]
    )
    
print(dataXyzNonNorm[0][:])
print(dataAnglesNonNorm[0][:])

[[0.29337959 0.44117639 0.17579085]
 [0.23152363 0.15508325 0.34810026]
 [0.39630531 0.49358194 0.2558147 ]
 [0.25610337 0.62959407 0.41140529]
 [0.45044206 0.10893607 0.10822496]
 [0.13995954 0.73991728 0.36394425]
 [0.38855935 0.42133052 0.41114762]
 [0.70381817 0.31579539 0.34827017]
 [0.47818133 0.39268708 0.24149685]
 [0.57109289 0.35384915 0.38416792]]
[[5.75326713e+00 7.34528794e-01 5.77920876e+00 6.23685501e+00
  6.01341752e+00]
 [5.31768008e+00 2.15816233e-01 5.62660411e+00 6.17342912e+00
  5.97828979e+00]
 [5.71231188e+00 2.95993331e-01 6.15227576e+00 6.27070177e+00
  6.22287517e+00]
 [5.27040923e+00 1.26911606e-01 2.55464204e-01 6.27393497e+00
  8.92010687e-02]
 [6.04966116e+00 6.26825248e+00 5.73836329e+00 6.26902438e+00
  6.04768122e+00]
 [5.08072494e+00 2.16930569e-01 4.03885116e-01 6.27282587e+00
  1.28066247e-01]
 [5.47113513e+00 5.73812064e-03 3.90898017e-04 6.26931474e+00
  6.28256428e+00]
 [5.82482011e+00 5.25704335e+00 1.15160900e+00 9.87558964e-03
  1.49501360e-01]

In [ ]:
def calc_T_numpy(q):

    c0 = np.cos(q[0])
    c1 = np.cos(q[1])
    c2 = np.cos(q[2])
    c3 = np.cos(q[3])
    c4 = np.cos(q[4])
    
    s0 = np.sin(q[0])
    s1 = np.sin(q[1])
    s2 = np.sin(q[2])
    s3 = np.sin(q[3])
    s4 = np.sin(q[4])
    
    return np.array([0.208*((-s1*c0*c2 - s2*c0*c1)*c3 + s0*s3)*s4 + 
                      0.208*(-s1*s2*c0 + c0*c1*c2)*c4 - 0.299*s1*s2*c0 - 
                      0.3*s1*c0 + 0.299*c0*c1*c2 + 0.06*c0*c1,
                     0.208*(-s1*s2 + c1*c2)*s4*c3 + 
                      0.208*(s1*c2 + s2*c1)*c4 + 
                      0.299*s1*c2 + 0.06*s1 + 0.299*s2*c1 + 0.3*c1 + 0.118,
                     0.208*((s0*s1*c2 + s0*s2*c1)*c3 + s3*c0)*s4 + 
                      0.208*(s0*s1*s2 - s0*c1*c2)*c4 + 0.299*s0*s1*s2 + 
                      0.3*s0*s1 - 0.299*s0*c1*c2 - 0.06*s0*c1], dtype='float')
    
    

def calc_J_numpy(q):

    c = np.cos(q)
    s = np.sin(q)

    c0 = c[0]
    c1 = c[1]
    c2 = c[2]
    c3 = c[3]
    c4 = c[4]


    s0 = s[0]
    s1 = s[1]
    s2 = s[2]
    s3 = s[3]
    s4 = s[4]

    s12  = np.sin(q[1] + q[2])
    c12  = np.cos(q[1] + q[2])

    J = np.zeros((3,5))

    J[0,0]= 0.208*((s0*s1*c2 + s0*s2*c1)*c3 + s3*c0)*s4 + 0.208*(s0*s1*s2 - s0*c1*c2)*c4 + 0.299*s0*s1*s2 + 0.3*s0*s1 - 0.299*s0*c1*c2 - 0.06*s0*c1
    J[0,1]= 0.208*(s1*s2*c0 - c0*c1*c2)*s4*c3 + 0.208*(-s1*c0*c2 - s2*c0*c1)*c4 - 0.299*s1*c0*c2 - 0.06*s1*c0 - 0.299*s2*c0*c1 - 0.3*c0*c1
    J[0,2]= 0.208*(s1*s2*c0 - c0*c1*c2)*s4*c3 + 0.208*(-s1*c0*c2 - s2*c0*c1)*c4 - 0.299*s1*c0*c2 - 0.299*s2*c0*c1
    J[0,3]= 0.208*(-(-s1*c0*c2 - s2*c0*c1)*s3 + s0*c3)*s4
    J[0,4]= 0.208*((-s1*c0*c2 - s2*c0*c1)*c3 + s0*s3)*c4 - 0.208*(-s1*s2*c0 + c0*c1*c2)*s4
    J[1,0]= 0
    J[1,1]= 0.208*(-s1*s2 + c1*c2)*c4 + 0.208*(-s1*c2 - s2*c1)*s4*c3 - 0.299*s1*s2 - 0.3*s1 + 0.299*c1*c2 + 0.06*c1
    J[1,2]= 0.208*(-s1*s2 + c1*c2)*c4 + 0.208*(-s1*c2 - s2*c1)*s4*c3 - 0.299*s1*s2 + 0.299*c1*c2
    J[1,3]= -0.208*(-s1*s2 + c1*c2)*s3*s4
    J[1,4]= 0.208*(-s1*s2 + c1*c2)*c3*c4 - 0.208*(s1*c2 + s2*c1)*s4
    J[2,0]= 0.208*((s1*c0*c2 + s2*c0*c1)*c3 - s0*s3)*s4 + 0.208*(s1*s2*c0 - c0*c1*c2)*c4 + 0.299*s1*s2*c0 + 0.3*s1*c0 - 0.299*c0*c1*c2 - 0.06*c0*c1
    J[2,1]= 0.208*(-s0*s1*s2 + s0*c1*c2)*s4*c3 + 0.208*(s0*s1*c2 + s0*s2*c1)*c4 + 0.299*s0*s1*c2 + 0.06*s0*s1 + 0.299*s0*s2*c1 + 0.3*s0*c1
    J[2,2]= 0.208*(-s0*s1*s2 + s0*c1*c2)*s4*c3 + 0.208*(s0*s1*c2 + s0*s2*c1)*c4 + 0.299*s0*s1*c2 + 0.299*s0*s2*c1
    J[2,3]= 0.208*(-(s0*s1*c2 + s0*s2*c1)*s3 + c0*c3)*s4
    J[2,4]= 0.208*((s0*s1*c2 + s0*s2*c1)*c3 + s3*c0)*c4 - 0.208*(s0*s1*s2 - s0*c1*c2)*s4
    
    return J


def calc_T(q):
    T = np.zeros(3, dtype=object)

    c0 = np.cos(q[0])
    c1 = np.cos(q[1])
    c2 = np.cos(q[2])
    c3 = np.cos(q[3])
    c4 = np.cos(q[4])
    
    s0 = np.sin(q[0])
    s1 = np.sin(q[1])
    s2 = np.sin(q[2])
    s3 = np.sin(q[3])
    s4 = np.sin(q[4])
    
    s12 = np.sin(q[1] + q[2])
    c12 = np.cos(q[1] + q[2])
    
    for index, j in np.ndenumerate(T):
        if(index == (0,)):
          T[index] = 0.208*s0*s3*s4 - 0.3*s1*c0 - 0.208*s4*s12*c0*c3 + 0.06*c0*c1 + 0.208*c0*c4*c12 + 0.299*c0*c12
        if(index == (1,)):
          T[index] =  0.06*s1 + 0.208*s4*c3*c12 + 0.208*s12*c4 + 0.299*s12 + 0.3*c1 + 0.118
        if(index == (2,)):
          T[index] =  0.3*s0*s1 + 0.208*s0*s4*s12*c3 - 0.06*s0*c1 - 0.208*s0*c4*c12 - 0.299*s0*c12 + 0.208*s3*s4*c0

    return gyrus.asoperator(T)

    
  
   
def calc_J(q):
    J = np.zeros((3,5), dtype=object)
    c = np.cos(q)
    s = np.sin(q)

    c0 = c[0]
    c1 = c[1]
    c2 = c[2]
    c3 = c[3]
    c4 = c[4]

    s0 = s[0]
    s1 = s[1]
    s2 = s[2]
    s3 = s[3]
    s4 = s[4]

    s12  = np.sin(q[1] + q[2])
    c12  = np.cos(q[1] + q[2])

    for index, j in np.ndenumerate(J):
        if(index == (0,0)):
          J[index] =  0.3*s0*s1 + 0.208*s0*s4*s12*c3 - 0.06*s0*c1 - 0.208*s0*c4*c12 - 0.299*s0*c12 + 0.208*s3*s4*c0
        if(index == (0,1)):
          J[index] =  -(0.06*s1 + 0.208*s4*c3*c12 + 0.208*s12*c4 + 0.299*s12 + 0.3*c1)*c0
        if(index == (0,2)):
          J[index] =  -(0.208*s4*c3*c12 + 0.208*s12*c4 + 0.299*s12)*c0
        if(index == (0,3)):
          J[index] = 0.208*(s0*c3 + s3*s12*c0)*s4
        if(index == (0,4)):
          J[index] = 0.208*(s0*s3 - s12*c0*c3)*c4 - 0.208*s4*c0*c12
        if(index == (1,0)):
          J[index] = 0 * s12  # need to only zero
        if(index == (1,1)):
          J[index] =  -0.3*s1 - 0.208*s4*s12*c3 + 0.06*c1 + 0.208*c4*c12 + 0.299*c12
        if(index == (1,2)):
          J[index] = -0.208*s4*s12*c3 + 0.208*c4*c12 + 0.299*c12
        if(index == (1,3)):
          J[index] = -0.208*s3*s4*c12
        if(index == (1,4)):
          J[index] = -0.208*s4*s12 + 0.208*c3*c4*c12
        if(index == (2,0)):
          J[index] = -0.208*s0*s3*s4 + 0.3*s1*c0 + 0.208*s4*s12*c0*c3 - 0.06*c0*c1 - 0.208*c0*c4*c12 - 0.299*c0*c12
        if(index == (2,1)):
          J[index] = (0.06*s1 + 0.208*s4*c3*c12 + 0.208*s12*c4 + 0.299*s12 + 0.3*c1)*s0
        if(index == (2,2)):
          J[index] = (0.208*s4*c3*c12 + 0.208*s12*c4 + 0.299*s12)*s0
        if(index == (2,3)):
          J[index] = -0.208*(s0*s3*s12 - c0*c3)*s4
        if(index == (2,4)):
          J[index] =  0.208*(s0*s12*c3 + s3*c0)*c4 + 0.208*s0*s4*c12
    #return gyrus.asoperator(J)
    return J

In [ ]:
def evaluate(A, A_hat,evaluate_all=False):
    Ainv = A #np.linalg.inv(A)
    if not evaluate_all:
      nrmse = rms(Ainv - A_hat) / rms(Ainv)
      print(f"Normalized RMSE: {(100 * nrmse).round(1)}%")

      vlim = np.max(np.abs([Ainv, A_hat]))
      cmap = cm.get_cmap("bwr")
      im = cm.ScalarMappable(cmap=cmap, norm=Normalize(-vlim, vlim))

      fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)
      axes[0].set_title("Ground Truth ($A$)")
      axes[0].imshow(Ainv, vmin=-vlim, vmax=vlim, cmap=cmap)
      axes[1].set_title(r"Approximation ($\tilde{A}$)")
      axes[1].imshow(A_hat, vmin=-vlim, vmax=vlim, cmap=cmap)
      axes[2].set_title(r"Error ($A - \tilde{A}$)")
      axes[2].imshow(Ainv - A_hat, vmin=-vlim, vmax=vlim, cmap=cmap)
      fig.colorbar(im, ax=axes.ravel().tolist())
      fig.show()

    euc = np.sqrt(np.sum((Ainv - A_hat)**2))
    if not evaluate_all: print("Distance from target im Meters: ", euc)
    return euc

def evaluate_all(A_list, A_hat_list):
  sum = 0
  for i in range(len(A_list)):
    sum += evaluate(A=A_list[i],A_hat=A_hat_list[i],evaluate_all=True )
  return sum / len(A_list)

def calc_T_for_data(A, data):
  data2 = np.zeros((data.shape[0],A.size), dtype=object)
  for i in range(data.shape[0]):
    data2[i] = calc_T_numpy(data[i,:])
  return data2

def calc_euc_data(A, data2):
  data3 = np.zeros(data2.shape[0], dtype=object)
  for i in range(data2.shape[0]):
    data3[i] = np.sqrt(np.sum((A - data2[i])**2)) 
  return data3

def plot_simulation(A, data, q):
  ''' plot q '''
  '''
  data = np.asarray(data).reshape(-1, q.size)
  qinv = q.flatten() # np.linalg.inv(A).flatten()
  plt.figure()
  for i in range(q.size):
      (line,) = plt.plot(data[:, i])
      plt.hlines(qinv[i], 0, len(data) - 1, color=line.get_color(), linestyle="--")
  plt.xlabel("Time-step")
  plt.show()
  '''
  ''' plot xyz '''
  data = calc_T_for_data(A, data)
  data = np.asarray(data).reshape(-1, A.size)
  Ainv = A.flatten() # np.linalg.inv(A).flatten()
  plt.figure()
  for i in range(A.size):
      (line,) = plt.plot(data[:, i])
      plt.hlines(Ainv[i], 0, len(data) - 1, color=line.get_color(), linestyle="--")
  plt.xlabel("Time-step")
  plt.show()
  
  ''' plot euc '''
  data = calc_euc_data(A, data)
  data = np.asarray(data)
  Ainv = A.flatten() # np.linalg.inv(A).flatten()
  plt.figure()
  (line,) = plt.plot(data)
  plt.xlabel("Time-step")
  plt.show()


In [ ]:
def gradient(A, M):
    """Compute the gradient of M approximating inv(A)."""
    I = np.eye(A.shape[1])
    return 2 * (M @ A - I) @ A.T


def gyrus_inverse(J, J_pinv_hat, dt, synapse=None):
    """Compute the inverse of J by gradient descent from J_pinv_hat."""
    return J_pinv_hat.integrate_fold(
        integrand=lambda J_pinv_hat: -dt * gradient(J, J_pinv_hat) / 1e-3,
        synapse=synapse,
    )


def gyrus_calc(A, q_hat, dt, synapse=None):
    """Compute q according to Jacobian matrix."""
    J = gyrus.stimuli(calc_J(q_hat)).configure(
            n_neurons=1000,
            input_magnitude=2,
            seed=0)
    
    J_pinv_hat = gyrus.stimuli(np.zeros_like(calc_J(q_hat)).T)

    ## np.linalg.pinv(calc_J(q_hat)) ##
    J_pinv = gyrus_inverse(J, J_pinv_hat, dt)

    return q_hat.integrate_fold(
        integrand=lambda q_hat: dt * J_pinv.dot(A-calc_T(q_hat)) / 1e-3,
        synapse=synapse,
    )
    

In [ ]:
def go(A, q, dt, t, output_synapse, n_neurons, neuron_type, use_intercepts, use_encoders, max_rates=None):
    op = gyrus_calc(
        A=gyrus.stimuli(A).configure(
            n_neurons=n_neurons,
            input_magnitude=2,
            seed=0,
        ),
        q_hat=gyrus.stimuli(np.zeros_like(q)),
        dt=dt,
    ).filter(output_synapse)

    with nengo.Network() as model:

        model.config[nengo.Ensemble].neuron_type = neuron_type
        model.config[nengo.Connection].solver = nengo.solvers.Lstsq()
        
        if max_rates:
           model.config[nengo.Ensemble].max_rates = nengo.dists.Choice([max_rates])

        accessor = gyrus.probe(op.make())

        for ens in model.all_ensembles:
          if use_intercepts:
            ens.intercepts =  get_intercepts(ens.n_neurons, ens.dimensions)
          if use_encoders:
            ens.encoders = encoders_dist.sample(ens.n_neurons, ens.dimensions) 

    with tf.device("/cpu:0"):
    #with nengo_dl.Simulator(model) as sim:
        with nengo.Simulator(model, optimize=False) as sim:
          sim.run(t)
        return model, accessor(sim)

In [ ]:
''' RegularSpiking ''' # with solver
q_hat_list = list()
A_hat_list = list()
q_list = list()
A_list = list()

for i in range(num_of_blocks):
  for j in range(num_of_points_in_block):

    A = dataXyzNonNorm[i][j]
    q = dataAnglesNonNorm[i][j]

    A_list.append(A)
    q_list.append(q)

    model, data = go(
        A=A,
        q=q,
        dt=0.005,
        t=8,
        max_rates=1000,
        output_synapse=0.01,
        n_neurons = 10,
        neuron_type = nengo.RegularSpiking(gyrus.Parabola()),
        use_intercepts = False, # 25 35 45
        use_encoders = False, 
    )


    out = np.asarray(data).squeeze(axis=-1)

    q_hat = out[..., -1]
    A_hat = calc_T_numpy(q_hat)

    A_hat_list.append(A_hat)
    q_hat_list.append(q_hat)



HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

In [ ]:
data_dict =   {'A_list' : A_list,
              'A_hat_list': A_hat_list,
              'q_list': q_list,
              'q_hat_list': q_hat_list,
              
             }
             

import pickle
with open('35-45-55_20_points_all_10-80_gyrus_8sec_v6_10n.pkl','wb') as f:
    pickle.dump(data_dict, f)

In [ ]:
print("avg distance from target in meters= ",evaluate_all(A_list, A_hat_list))

avg distance from target in meters=  0.02261032271755551


In [ ]:
''' Direct '''

model, data = go(
    A=A,
    q=q,
    dt=0.005,
    t=2.5,
    max_rates=None,
    output_synapse=0.005,
    n_neurons=1,
    neuron_type = nengo.Direct(),
    use_intercepts = False, 
    use_encoders = False,
)

out = np.asarray(data).squeeze(axis=-1)

q_hat = out[..., -1]
A_hat = calc_T_numpy(q_hat)

plot_simulation(A, np.moveaxis(out, -1, 0), q)  # move time to first dimension
evaluate(np.array([A,A,A],dtype='float'), np.array([A_hat,A_hat,A_hat],dtype='float'))